In [11]:
import pandas as pd
import requests
import time

In [12]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://tiki.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

In [13]:
URL = "https://api.tiki.vn/raiden/v2/menu-config?platform=desktop"

In [14]:
response = requests.get(URL, headers=HEADERS)
data = response.json()

In [19]:
groups = data["menu_block"]["items"]
group_list = []

for group in groups:
    text = group["text"]
    link = group["link"]
    # Extract category ID from the link
    group_id = link.split("/")[-1][1:]  
    group_list.append([group_id, text])

group_df = pd.DataFrame(group_list, columns=["Group_ID", "Group_Name"])
filtered_df = group_df[group_df['Group_ID'].isin([8322, 17166])]

# Print filtered_df to verify the result
print(filtered_df)


Empty DataFrame
Columns: [Group_ID, Group_Name]
Index: []


### Choosing the Group ID: 8322 and 17166

In [18]:
# Filtering 8322 and 17166 
filtered_df = group_df[group_df['Group_ID'].isin([8322, 17166])]
filtered_df

,Group_ID,Group_Name


In [17]:
parent_category_list = []
child_category_list = []

for group_id in group_df["Group_ID"]:
    parent_url = f"https://tiki.vn/api/v2/categories?parent_id={group_id}"   
    parent_response = requests.get(parent_url, headers=HEADERS)
    
    if parent_response.status_code == 200:
        parent_data = parent_response.json()
        if not parent_data["data"]:
            parent_category_list.append([group_id, group_df[group_df["Group_ID"] == group_id]["Name"].item(), None, None])
        else:
            for parent_category in parent_data["data"]:
                parent_id = parent_category["id"]
                parent_name = parent_category["name"]
                
                # Get group name 
                group_name = group_df[group_df["Group_ID"] == group_id]["Name"].item()
                
                parent_category_list.append([group_id, group_name, parent_id, parent_name])
                
                # Now fetch child categories
                child_url = f"https://tiki.vn/api/v2/categories?parent_id={parent_id}" 
                child_response = requests.get(child_url, headers=HEADERS)
                
                if child_response.status_code == 200:
                    child_data = child_response.json()
                    if not child_data["data"]:
                        child_category_list.append([group_id, group_name, parent_id, parent_name, None, None])
                    else:
                        for child_category in child_data["data"]:
                            child_id = child_category["id"]
                            child_name = child_category["name"]
                            
                            child_category_list.append([group_id, group_name, parent_id, parent_name, child_id, child_name])

parent_category = pd.DataFrame(parent_category_list, columns=["Group_ID", "Group_Name", "Parent_ID", "Parent_Name"])
child_category = pd.DataFrame(child_category_list, columns=["Group_ID", "Group_Name", "Parent_ID", "Parent_Name", "Child_ID", "Child_Name"])

KeyError: 'Name'

In [ ]:
import pandas as pd

# Assuming you already have group_df

# Create DataFrame from the given data
data = {
    'Group_ID': [8322, 1883, 1789, 2549, 1815, 1882, 1520, 8594, 931, 4384, 
                 1975, 915, 17166, 1846, 1686, 4221, 1703, 1801, 27498, 
                 44792, 8371, 6000, 11312, 976, 27616],
    'Group_Name': ['Nhà Sách Tiki', 'Nhà Cửa - Đời Sống', 'Điện Thoại - Máy Tính Bảng', 
                   'Đồ Chơi - Mẹ & Bé', 'Thiết Bị Số - Phụ Kiện Số', 'Điện Gia Dụng', 
                   'Làm Đẹp - Sức Khỏe', 'Ô Tô - Xe Máy - Xe Đạp', 'Thời trang nữ', 
                   'Bách Hóa Online', 'Thể Thao - Dã Ngoại', 'Thời trang nam', 
                   'Cross Border - Hàng Quốc Tế', 'Laptop - Máy Vi Tính - Linh kiện', 
                   'Giày - Dép nam', 'Điện Tử - Điện Lạnh', 'Giày - Dép nữ', 
                   'Máy Ảnh - Máy Quay Phim', 'Phụ kiện thời trang', 'NGON', 
                   'Đồng hồ và Trang sức', 'Balo và Vali', 'Voucher - Dịch vụ', 
                   'Túi thời trang nữ', 'Túi thời trang nam']
}

group_df = pd.DataFrame(data)

# Select rows with Group_ID 8322 and 17166
filtered_df = group_df[group_df['Group_ID'].isin([8322, 17166])]

# Print filtered_df to verify the result
print(filtered_df[['Group_ID', 'Group_Name']])


    Group_ID                   Group_Name
0       8322                Nhà Sách Tiki
12     17166  Cross Border - Hàng Quốc Tế
